# Deep Learning and Computer Chess

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import chess
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv('../output/evaluations.csv')
df.head()

,FEN,Evaluation,Category
0,r3k3/p4n1p/3p1Bq1/3P4/2p2P2/7P/P3Q1P1/5R1K b q...,5.2,White is winning
1,r3k3/p4n1p/3p1Bq1/3P4/2p2P2/7P/P3Q1P1/5R1K b q...,5.2,White is winning
2,4Q3/7k/2p4p/1p1qP1p1/3B1nR1/2P4P/3r2P1/6K1 w -...,0.0,Even
3,4Q3/7k/2p4p/1p1qP1p1/3B1nR1/2P4P/3r2P1/6K1 w -...,0.0,Even
4,2k4r/ppp2p2/2b2B2/7p/6pP/2P4P/PP3Nq1/R5K1 w - ...,0.0,Even


In [20]:
df.shape

(76916, 3)

In [21]:
df.isnull().sum()

FEN           0
Evaluation    0
Category      0
dtype: int64

In [22]:
def fen_to_bitstring(fen):
    board = chess.Board(fen)

    piece_types = {
        'P': 0,
        'N': 1,
        'B': 2,
        'R': 3,
        'Q': 4,
        'K': 5,
        'p': 6,
        'n': 7,
        'b': 8,
        'r': 9,
        'q': 10,
        'k': 11
    }

    bitboard = np.zeros(768, dtype=int)

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            piece_index = piece_types[piece.symbol()]
            bitboard[square * 12 + piece_index] = 1

    side_to_move = 1 if board.turn == chess.WHITE else 0

    castling_rights = [
        1 if board.has_kingside_castling_rights(chess.WHITE) else 0,
        1 if board.has_queenside_castling_rights(chess.WHITE) else 0,
        1 if board.has_kingside_castling_rights(chess.BLACK) else 0,
        1 if board.has_queenside_castling_rights(chess.BLACK) else 0
    ]

    bitstring = np.concatenate((bitboard, [side_to_move], castling_rights))

    return bitstring


In [23]:
df['bitstring'] = df['FEN'].apply(lambda x: fen_to_bitstring(x))
df.head()

,FEN,Evaluation,Category,bitstring
0,r3k3/p4n1p/3p1Bq1/3P4/2p2P2/7P/P3Q1P1/5R1K b q...,5.2,White is winning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,r3k3/p4n1p/3p1Bq1/3P4/2p2P2/7P/P3Q1P1/5R1K b q...,5.2,White is winning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4Q3/7k/2p4p/1p1qP1p1/3B1nR1/2P4P/3r2P1/6K1 w -...,0.0,Even,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4Q3/7k/2p4p/1p1qP1p1/3B1nR1/2P4P/3r2P1/6K1 w -...,0.0,Even,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,2k4r/ppp2p2/2b2B2/7p/6pP/2P4P/PP3Nq1/R5K1 w - ...,0.0,Even,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
class DBN(nn.Module):
    def __init__(self):
        super(DBN, self).__init__()
        self.dbn = nn.Sequential(
            nn.Linear(773, 600),
            nn.ReLU(),
            nn.Linear(600, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
        )

    def forward(self, x):
        return self.dbn(x)

In [10]:
class DeepChess(nn.Module):
    def __init__(self):
        super(DeepChess, self).__init__()
        self.branch = nn.DBN()

        self.merge_layers = nn.Sequential(
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )

    def forward(self, x1, x2):
        x1 = self.branch(x1)
        x2 = self.branch(x2)

        x = torch.cat((x1, x2), dim=1)

        return self.merge_layers(x)